# Page for trying out the data

In [1]:
import pandas as pd
import time, os, shutil
import nltk
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np  
from sklearn.cluster import KMeans  
from sklearn.cluster import DBSCAN


W0511 16:00:32.411990  8600 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [4]:
#df1 = pd.read_csv('amazon_reviews_us_Mobile_Electronics_v1_00.tsv',sep="\t", error_bad_lines=False)
# bad lines exist......
#data_source = 'amazon_reviews_us_Mobile_Electronics_v1_00.tsv'
#data_source='amazon_reviews_us_PC_v1_00.tsv.gz'
#data_source='amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
data_source = 'amazon_reviews_us_Electronics_v1_00.tsv.gz'
#data_source = 'amazon_reviews_us_Major_Appliances_v1_00.tsv.gz'
df1 = pd.read_csv(data_source,sep="\t", error_bad_lines=False)

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

### DATA COLUMNS:
marketplace       - 2 letter country code of the marketplace where the review was written.  
customer_id       - Random identifier that can be used to aggregate reviews written by a single author.  
review_id         - The unique ID of the review.  
product_id        - The unique Product ID the review pertains to. In the multilingual dataset the reviews
                    for the same product in different countries can be grouped by the same product_id.  
product_parent    - Random identifier that can be used to aggregate reviews for the same product.  
product_title     - Title of the product.  
product_category  - Broad product category that can be used to group reviews  
                    (also used to group the dataset into coherent parts).  
star_rating       - The 1-5 star rating of the review.  
helpful_votes     - Number of helpful votes.  
total_votes       - Number of total votes the review received.  
vine              - Review was written as part of the Vine program.  
verified_purchase - The review is on a verified purchase.  
review_headline   - The title of the review.  
review_body       - The review text.  
review_date       - The date the review was written.  

Filter out the ones not versified and get a subset of the data

In [5]:
df1 = df1.loc[df1['verified_purchase']=='Y',['product_title', 'helpful_votes','review_headline', 'review_body']]

In [6]:
df2 = df1.loc[:,['review_headline', 'review_body']]

In [7]:
def clean_html(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

In [8]:
df2['summary'] = df2.review_headline
df2['text'] = df2.review_body
df2 = df2.loc[:,['text','summary']].reset_index(drop='True')

In [9]:
df2 = pd.DataFrame(df2)

In [10]:
df2[:5]

,text,summary
0,As described.,Five Stars
1,It works as advertising.,It works as advertising.
2,Works pissa,Five Stars
3,Did not work at all.,One Star
4,Works well. Bass is somewhat lacking but is pr...,Overall pleased with the item


In [11]:
df2['c_text'] = df2['text'].apply(lambda x: clean_html(str(x)))

In [12]:
df2['c_summary'] = df2['summary'].apply(lambda x: clean_html(str(x)))

In [13]:
df2 = df2.loc[:,['c_text','c_summary']]

In [14]:
df2['text'] = df2['c_text']
df2['summary'] = df2['c_summary']
df2 = df2.loc[:,['text','summary']]

In [15]:
df2['len_text'] = df2['text'].apply(lambda x: len(x.split()))
df2['len_summary'] = df2['summary'].apply(lambda x: len(x.split()))
df2[:10]

,text,summary,len_text,len_summary
0,As described.,Five Stars,2,2
1,It works as advertising.,It works as advertising.,4,4
2,Works pissa,Five Stars,2,2
3,Did not work at all.,One Star,5,2
4,Works well. Bass is somewhat lacking but is pr...,Overall pleased with the item,14,5
5,The quality on these speakers is insanely good...,Five Stars,19,2
6,Wish I could give this product more than five ...,Lifesaver,11,1
7,works great,Five Stars,2,2
8,Great sound and compact. Battery life seems go...,Four Stars,12,2
9,It works well~~~,Four Stars,3,2


In [ ]:
#df2.to_csv('output/sub-set/origin_PC.csv',index=False)
#df2.to_csv('output/sub-set/origin_elec.csv',index=False)

In [16]:
df2.len_text.describe(include="all").apply(lambda x: format(x, 'f'))

count    2597626.000000
mean          56.189222
std           87.952698
min            1.000000
25%           17.000000
50%           31.000000
75%           65.000000
max         8106.000000
Name: len_text, dtype: object

In [17]:
df2.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

count    2597626.000000
mean           4.291171
std            9.218908
min            0.000000
25%            2.000000
50%            3.000000
75%            5.000000
max         9261.000000
Name: len_summary, dtype: object

In [18]:
df3 = df2[df2.len_summary>=10].reset_index(drop=True)

In [19]:
df3 = df3[df3.len_text>=50].reset_index(drop=True)

In [20]:
df3 = df3[df3.len_text<=300].reset_index(drop=True)

In [21]:
df3.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

count    92567.000000
mean        12.996770
std         26.573464
min         10.000000
25%         10.000000
50%         12.000000
75%         14.000000
max       7299.000000
Name: len_summary, dtype: object

In [22]:
df3.len_text.describe(include="all").apply(lambda x: format(x, 'f'))

count    92567.000000
mean       108.805363
std         57.387654
min         50.000000
25%         65.000000
50%         89.000000
75%        136.000000
max        300.000000
Name: len_text, dtype: object

In [23]:
df3.to_csv('output/sub-set/word_lim_50_300_elec2.csv',index=False)

Reload and start from here

In [24]:
df = pd.read_csv('output/sub-set/word_lim_50_300_elec2.csv')

In [25]:
df.len_text.describe(include="all").apply(lambda x: format(x, 'f'))                                                          

count    92567.000000
mean       108.805363
std         57.387654
min         50.000000
25%         65.000000
50%         89.000000
75%        136.000000
max        300.000000
Name: len_text, dtype: object

In [ ]:
df.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))

In [26]:
from bs4 import BeautifulSoup
from html import unescape

def clean_text(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) ## remove URL
    text = BeautifulSoup(unescape(text), 'lxml').text ## remove strange characters like &amp; 
    text = re.sub(r'[_"\-%()|+&=*%#@\[\]/\r\n\t]', '', text) ## remove other unwanted characters
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text) ## if more than 2 consecutive e or o, remove them
    text = re.sub(r"([^eoEO])\1\1+",r"\1",one) ## if more than 2 consecutive other characters, remove them
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text

In [27]:
df['text'] = df['text'].apply(lambda x: clean_text(x))

In [28]:
df['summary'] = df['summary'].apply(lambda x: clean_text(x))

In [29]:
df['len_text'] = df['text'].apply(lambda x: len(x.split()))
df['len_summary'] = df['summary'].apply(lambda x: len(x.split()))

In [30]:
df.len_text.describe(include="all").apply(lambda x: format(x, 'f'))   

count    92567.000000
mean       108.471766
std         57.183953
min          3.000000
25%         65.000000
50%         89.000000
75%        135.000000
max        300.000000
Name: len_text, dtype: object

In [31]:
df.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))   

count    92567.000000
mean        12.862813
std         20.033188
min          0.000000
25%         10.000000
50%         12.000000
75%         14.000000
max       5417.000000
Name: len_summary, dtype: object

In [32]:
df = df.loc[df.len_summary>=10]
df = df.loc[df.len_text>=50]
df = df.loc[df.len_summary<=20]
df = df.loc[df.len_text<=200]

In [33]:
df = df.reset_index(drop=True)
df

,text,summary,len_text,len_summary
0,It does not copy CDR s to USB as advertised. M...,It does not copy CDR s to USB as advertised .,51,11
1,Appears to work just great. Used it to connect...,Easy installation and setup use USB on your TV...,79,14
2,I got these and used them a couple o times to ...,I got these and used them a couple o times .,79,11
3,"I'm really not qualified to review headphones,...",but some members on headfi gave them really go...,58,19
4,These do not hold as long of a charge as the g...,These do not hold as long of a charge as .,70,11
5,Ordered to hookup a backup camera. This thing ...,Ordered to hookup a backup camera. This thing ...,55,10
6,I bought this to replace a lost neoprene case ...,I bought this to replace a lost neoprene case ...,61,11
7,"When I bought this speaker, I had one request....",Which I recommend to anyone with a relative li...,135,10
8,after a lightening strike wiped out my old rec...,Loving it! But that display? 1980's are callin...,198,12
9,Product is just okay! Volume level is not very...,"Volume level is not very good, I am used to go...",77,12


In [35]:
df.to_csv('output/amazon/clean_electronic.csv',index=False)

In [36]:
########################## re-start work
df = pd.read_csv('output/amazon/clean_electronic.csv')

In [37]:
from better_profanity import profanity

In [38]:
df['text'] = df['text'].apply(lambda x: profanity.censor(x))
df['summary'] = df['summary'].apply(lambda x: profanity.censor(x))

In [39]:
def clean_text_again(text):
    text = re.sub(r"([!$*.,?:])\1\1+",r"\1",text) ## if more than 2 consecutive characters, remove until only one left
    text = re.sub(r"([,.?!:])",r" \1 ",text) #s = "This must not b3 delet3d, but the number at the end yes 134411"
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text.lower()

In [40]:
str1 = 'AAAAAaaa......what??? no *****!!!!'
clean_text_again(str1)

'aaaaaaaa . what ? no * ! '

In [41]:
df['text'] = df['text'].apply(lambda x: clean_text_again(x))
df['summary'] = df['summary'].apply(lambda x: clean_text_again(x))

In [42]:
df['len_text'] = df['text'].apply(lambda x: len(x.split()))
df['len_summary'] = df['summary'].apply(lambda x: len(x.split()))

In [43]:
df.len_summary.describe(include="all").apply(lambda x: format(x, 'f'))   

count    78178.000000
mean        13.289826
std          2.767031
min         10.000000
25%         11.000000
50%         13.000000
75%         15.000000
max         33.000000
Name: len_summary, dtype: object

In [44]:
df.len_text.describe(include="all").apply(lambda x: format(x, 'f'))   

count    78178.000000
mean       104.804613
std         43.230639
min         50.000000
25%         69.000000
50%         92.000000
75%        131.000000
max        261.000000
Name: len_text, dtype: object

In [45]:
df = df.loc[df.len_summary>=10]
df = df.loc[df.len_text>=50]
df = df.loc[df.len_summary<=20]
df = df.loc[df.len_text<=200]
df = df.reset_index(drop=True)

In [ ]:
s = "This must not b3 delet3d, but the number at the end yes 134411"
re.sub(r"\d+", "#", s)

In [46]:
### replace all numbers to #
df['text'] = df['text'].apply(lambda x: re.sub(r"\d+", "#", x))
df['summary'] = df['summary'].apply(lambda x: re.sub(r"\d+", "#", x))

In [47]:
df[:10]

,text,summary,len_text,len_summary
0,it does not copy cdr s to usb as advertised . ...,it does not copy cdr s to usb as advertised .,59,11
1,appears to work just great . used it to connec...,easy installation and setup use usb on your tv...,83,14
2,i got these and used them a couple o times to ...,i got these and used them a couple o times .,89,11
3,i'm really not qualified to review headphones ...,but some members on headfi gave them really go...,65,19
4,these do not hold as long of a charge as the g...,these do not hold as long of a charge as .,77,11
5,ordered to hookup a backup camera . this thing...,ordered to hookup a backup camera . this thing...,59,11
6,i bought this to replace a lost neoprene case ...,i bought this to replace a lost neoprene case ...,67,11
7,"when i bought this speaker , i had one request...",which i recommend to anyone with a relative li...,155,10
8,product is just okay ! volume level is not ver...,"volume level is not very good , i am used to g...",86,13
9,"well , i purchased this masione® surge protect...",no longer labeled as surge protection after i ...,94,11


In [48]:
df.to_csv('output/amazon/clean_electronic2.csv',index=False)